* `datasets: get datasets and data loading functions for computer vision here`
* `models: get pretrained model that can leverage for own problem`
* `transforms: manipulating ur vision data to be suitable for use of DP model`
* `Dataset: Base dataset class for Pytorch`
* `DataLoader: create a python iterable over dataset`

In [64]:
import torch
from torch import nn

# Torchvision
import torchvision
from torchvision import datasets
from torchvision import transforms
from torchvision.transforms import ToTensor

# plt
import matplotlib.pyplot as plt

In [65]:
train_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=torchvision.transforms.ToTensor(), # convert to tensor
    target_transform=None
)

In [66]:
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
    target_transform=None
)

In [67]:
image, label = train_data[0]

In [68]:
class_names = train_data.classes
class_names

['T-shirt/top',
 'Trouser',
 'Pullover',
 'Dress',
 'Coat',
 'Sandal',
 'Shirt',
 'Sneaker',
 'Bag',
 'Ankle boot']

In [69]:
image.shape # batch_size=None, color, w, h

torch.Size([1, 28, 28])

### Prepare Dataloader
* tunrn data into mini batches
* Why - more computational efficient, hardware may not be able to store in memory at 60000 images in one hit. -> break down to 32 images at a time (batch size of 32)
* More chances to update its gradients per epoch

In [70]:
from torch.utils.data import DataLoader
# batch_size: samples per batch

In [71]:
BATCH_SIZE = 32
# DataLoader for Dataset
train_dataloader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(dataset=test_data, batch_size=BATCH_SIZE, shuffle=False)

In [72]:
train_dataloader, test_dataloader

(<torch.utils.data.dataloader.DataLoader at 0x7c0ae2e86320>,
 <torch.utils.data.dataloader.DataLoader at 0x7c0ae2e87c70>)

In [73]:
# inside training dataloader
train_feats_batch, train_labels_batch = next(iter(train_dataloader))
train_feats_batch.shape, train_labels_batch.shape

(torch.Size([32, 1, 28, 28]), torch.Size([32]))

# Base Line Model

Base line model - a simple model u'll try and improve upon with subsequent model - Start simply and add complexity when necessary

Flatten: https://www.superdatascience.com/blogs/convolutional-neural-networks-cnn-step-3-flattening

In [74]:
# Flatten layer
flatten_model = nn.Flatten()

# Get a single sample
x = train_feats_batch[0]
# 1, 28, 28 - color, H, W
# Flatten sample
output = flatten_model(x)
# color, H*W
print(f"before flattening {x.shape} After {output.shape}")

before flattening torch.Size([1, 28, 28]) After torch.Size([1, 784])


In [75]:
class Model0(nn.Module):
  def __init__(self, input_shape, hidden_units, output_shape):
    super().__init__()
    # define layer stack with Sequential
    self.layer_stack = nn.Sequential(
        # Flatting from tensor -> vector (including features)
        nn.Flatten(), # flatten layer
        nn.Linear(in_features=input_shape, out_features=hidden_units), # adjust output shape -> if dont have Flatten -> adjust in_features=28
        nn.ReLU(),
        # outout -> output_shape = 10 classes if dont have Flatten [1, 1, 28, 10]
        nn.Linear(in_features=hidden_units, out_features=output_shape), # 2 Linear have to link together Li1 (out) -> Li2 (in)
        nn.ReLU()
    )

  def forward(self, x):
    return self.layer_stack(x)

In [76]:
device = "cpu"

In [77]:
# Setup with input params
model0 = Model0(input_shape=784, hidden_units=20, output_shape=len(class_names)).to(device)

In [78]:
model0

Model0(
  (layer_stack): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=784, out_features=20, bias=True)
    (2): ReLU()
    (3): Linear(in_features=20, out_features=10, bias=True)
    (4): ReLU()
  )
)

In [79]:
x = torch.randn([1, 1, 28, 28]) # batch_size, color, H, W

In [80]:
# Passing x to model
model0(x).shape

torch.Size([1, 10])

# Creating loss function and Optimizer

* Loss function multi-classes `CrossEntropyLoss`
* Optimizer `SGD | Adam`
* Metric `Accuracy`

In [81]:
import requests
from pathlib import Path

In [82]:
req = requests.get("https://raw.githubusercontent.com/mrdbourke/pytorch-deep-learning/main/helper_functions.py")
with open("helper_functions.py", "wb") as f:
  f.write(req.content)

In [83]:
from helper_functions import accuracy_fn

In [84]:
# Setup loss and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=model0.parameters(), lr=0.01)

### Note
* Model's performance (loss and accuracy value)
* Time

In [85]:
from timeit import default_timer as timer

In [86]:
def train_time(start: float, end: float, device: torch.device = None):
  total_time = end - start
  print(f"Train time on {device}: {total_time:.3f} s")

### Training and test loop for batched data

1. Loop through epochs
2. Loop through batches, perform training steps, cal the train loss *per batch*
3. Loop through testing batches, perform testing steps, cal test loss *per batch*
4. Time it all

In [87]:
from tqdm.auto import tqdm

In [88]:
# start timer
train_time_start = timer()

# set the number of epochs
epochs = 3
for epoch in tqdm(range(epochs)):
  print(f"Epoch {epoch}")
  # training loss
  train_loss = 0
  # Add a loop through the training batches
  for batch, (X, y) in enumerate(train_dataloader):
    model0.train() # train
    # forward pass
    y_pred = model0(X) # X - input
    # Loss
    loss = loss_fn(y_pred, y)
    train_loss += loss

    # Optimizer zero grad
    optimizer.zero_grad()

    # Loss backward
    loss.backward()

    # Optimizer steps
    optimizer.step()

    if batch % 400 == 0:
      print(f"Looked at Batch {batch} and {batch*len(X)}/{len(train_dataloader.dataset)} samples")

  # devide train loss by length of train dataloader
  train_loss /= len(train_dataloader)

  # Test loss
  test_loss, test_acc = 0, 0
  model0.eval()

  with torch.inference_mode():
    for X_test, y_test in test_dataloader:
      test_pred = model0(X_test)
      test_loss += loss_fn(test_pred, y_test)
      test_acc += accuracy_fn(y_true=y_test, y_pred=test_pred.argmax(dim=1))

    test_loss /= len(test_dataloader)
    test_acc /= len(test_dataloader)
  print(f"Train loss: {train_loss: .4f} | test loss: {test_loss:.4f} test acc: {test_acc:.4f}")

# cal time
train_time_end = timer()
total_train_time = train_time(start=train_time_start, end=train_time_end, device=str(next(model0.parameters())))

  0%|          | 0/3 [00:00<?, ?it/s]

Epoch 0
Looked at Batch 0 and 0/60000 samples
Looked at Batch 400 and 12800/60000 samples
Looked at Batch 800 and 25600/60000 samples
Looked at Batch 1200 and 38400/60000 samples
Looked at Batch 1600 and 51200/60000 samples
Train loss:  1.2082 | test loss: 0.8968 test acc: 71.0363
Epoch 1
Looked at Batch 0 and 0/60000 samples
Looked at Batch 400 and 12800/60000 samples
Looked at Batch 800 and 25600/60000 samples
Looked at Batch 1200 and 38400/60000 samples
Looked at Batch 1600 and 51200/60000 samples
Train loss:  0.6229 | test loss: 0.5859 test acc: 79.3830
Epoch 2
Looked at Batch 0 and 0/60000 samples
Looked at Batch 400 and 12800/60000 samples
Looked at Batch 800 and 25600/60000 samples
Looked at Batch 1200 and 38400/60000 samples
Looked at Batch 1600 and 51200/60000 samples
Train loss:  0.5240 | test loss: 0.5320 test acc: 81.4597
Train time on Parameter containing:
tensor([[-0.0224, -0.0288,  0.0296,  ...,  0.0019,  0.0069,  0.0044],
        [ 0.0084,  0.0076,  0.0188,  ...,  0.004

  make predictions and get model results

In [89]:
def eval_model(model: torch.nn.Module, data_loader: torch.utils.data.DataLoader, loss_fn: torch.nn.Module, accuracy_fn):
  loss, acc = 0, 0
  model.eval()
  with torch.inference_mode():
    for X,y in data_loader:
      y_pred= model(X)

      loss += loss_fn(y_pred, y)
      acc += accuracy_fn(y_true=y, y_pred=y_pred.argmax(dim=1))

    loss /= len(data_loader)
    acc /= len(data_loader)

  return {"model_name": model.__class__.__name__,
          "model_loss": loss.item(),
          "model_acc": acc}

model0_rs = eval_model(model=model0, data_loader=test_dataloader,loss_fn=loss_fn, accuracy_fn=accuracy_fn)

In [90]:
model0_rs

{'model_name': 'Model0',
 'model_loss': 0.5319722890853882,
 'model_acc': 81.45966453674122}